In [1]:
import pandas as pd
import matplotlib.pyplot as plt

paths = {"CD1-E_no2 dataset, ALE method, GCN model": "C:/Users/ppaul/Documents/influence_on_ideas/data/CD1-E_no2/ALE_all_GCN_n_layers2_hidden_size256.csv",
        "CD1-E_no2 dataset, ALE method, GAT model": "C:/Users/ppaul/Documents/influence_on_ideas/data/CD1-E_no2/ALE_all_GAT_n_layers2_hidden_size256.csv", 
         "citations dataset, ALE method, GCN model": "C:/Users/ppaul/Documents/influence_on_ideas/data/citations/ALE_04.08_GCN_n_layers2_hidden_size256.csv",
         "citations dataset, ALE method, GAT model": "C:/Users/ppaul/Documents/influence_on_ideas/data/citations/ALE_04.08_GAT_n_layers2_hidden_size256.csv"}
#path_ale_gat = pd.read_csv("C:/Users/ppaul/Documents/influence_on_ideas/data/CD1-E_no2/ALE_GAT_n_layers2_hidden_size256.csv", names=['trial', 'k', 'max_bin_size', 'explanation_exact', 'time_exact', 'explanation_approximate', 'time_approximate'], header=None)
def read_data(path):

    if "cit" in path:
        df = pd.read_csv(
            path,
            names=['index', 'k', 'max_bin_size',  'explanation_approximate', 'time_approximate', 'explanation_exact', 'time_exact'], 
            header=None, index_col=None)
    else:
        df = pd.read_csv(
            path,
            index_col=None,
        )
    return df

In [4]:
import numpy as np
def chi2(df):

    chi2 = (df['explanation_exact']['mean'] - df['explanation_approximate']['mean'])**2 /(df['explanation_exact']['sem']**2 + df['explanation_approximate']['sem']**2)
    return chi2

def sem(x):
    return x.sem(ddof=0)

def calculate_chi2(path):
    df = read_data(path)
    grouped_df = df.groupby('index').agg({
        'explanation_exact': ['mean', sem],
        'explanation_approximate': ['mean', sem]
    })
    grouped_df['chi2'] = chi2(grouped_df)
    print(grouped_df)
    print(path, grouped_df['chi2'].sum())
    #mod = df.apply(lambda x: x['k']*x['max_bin_size'], axis=1).sum()/5
    #gt = df.set_index('index').apply(lambda x: x['explanation_exact']*x['max_bin_size']*x['k']/mod,axis=1).groupby(['index']).sum()
    #ap = df.set_index('index').apply(lambda x: x['explanation_approximate']*x['max_bin_size']*x['k']/mod,axis=1).groupby(['index']).sum()
    #grouped_df['explanation_exact']['mean'] = gt
    #grouped_df['explanation_approximate']['mean'] = ap
    #print(grouped_df, gt)
    #print(chi2(grouped_df).sum())


In [5]:
for path in paths.values():
    calculate_chi2(path)

      explanation_exact           explanation_approximate                chi2
                   mean       sem                    mean       sem          
index                                                                        
0             -0.000335  0.000235                0.000401  0.000463  2.014200
1              0.000357  0.000376                0.001551  0.000745  2.045627
2              0.002165  0.000454                0.005111  0.001102  6.105956
3              0.003339  0.000503                0.005578  0.001190  3.004068
4              0.005877  0.000518                0.008674  0.001251  4.269286
C:/Users/ppaul/Documents/influence_on_ideas/data/CD1-E_no2/ALE_all_GCN_n_layers2_hidden_size256.csv 17.439136383523113
      explanation_exact           explanation_approximate                chi2
                   mean       sem                    mean       sem          
index                                                                        
0             -0.016855

In [40]:
(gt-ap)**2/

index
0    4.029242e-07
1    2.381063e-11
2    1.527278e-06
3    9.330201e-07
4    1.326012e-06
dtype: float64

In [18]:
import numpy as np
df = read_data(paths["citations dataset, ALE method, GCN model"])
df["trial"] = np.floor([x/5 for x in df.index.to_list()])

In [32]:
grouped = df.groupby("trial").agg(
    {
        "k": lambda x: list(x)[0],
        "max_bin_size": lambda x: list(x)[0],
        "explanation_exact": list,
        "explanation_approximate": list,
    }
)
grouped = pd.concat(
    [
        grouped[["k", "max_bin_size", "explanation_exact"]].rename(
            {"explanation_exact": "explanation"}, axis=1
        ),
        grouped[["k", "max_bin_size", "explanation_approximate"]].rename(
            {"explanation_approximate": "explanation"}, axis=1
        ),
    ]
)
grouped["explanation"] = grouped["explanation"].apply(lambda x: np.array(x))


def goldstandard(df):
    mod = df.apply(lambda x: x["k"] * x["max_bin_size"], axis=1).sum()
    gt = df.apply(
        lambda x: x["explanation"] * x["max_bin_size"] * x["k"] / mod, axis=1
    ).sum()
    return gt


def root_mean_squared_error(x, y):
    return np.sqrt(np.mean((x - y) ** 2))


def statistic(df):
    return root_mean_squared_error(
        goldstandard(df.iloc[: int(len(df) / 2)]),
        goldstandard(df.iloc[int(len(df) / 2) :]),
    )


org = statistic(grouped)
stats_list = []
counter = 0
n = 10000
for i in range(n-1):
    # Shuffle the combined data
    val = statistic(grouped.sample(frac=1))
    stats_list.append(val)
    counter+=val<org
print(counter/n)


0.5797


In [ ]:
import numpy as np

def permutation_test(explanation_exact, explanation_approximate, n_permutations=10000):
    # Combine the data
    combined_data = np.concatenate([explanation_exact, explanation_approximate])
    
    # Calculate the observed difference in means
    observed_diff = np.mean(explanation_exact) - np.mean(explanation_approximate)
    
    # Initialize an array to store the differences from permutation
    permuted_diffs = np.zeros(n_permutations)
    
    # Perform permutations
    for i in range(n_permutations):
        # Shuffle the combined data
        np.random.shuffle(combined_data)
        
        # Split the permuted data
        perm_exact = combined_data[:len(explanation_exact)]
        perm_approx = combined_data[len(explanation_exact):]
        
        # Calculate the difference in means for this permutation
        permuted_diffs[i] = np.mean(perm_exact) - np.mean(perm_approx)
    
    # Calculate the p-value
    p_value = np.mean(np.abs(permuted_diffs) >= np.abs(observed_diff))
    
    return observed_diff, p_value, permuted_diffs

# Example usage:
# Assuming explanation_exact and explanation_approximate are numpy arrays containing the data
observed_diff, p_value, permuted_diffs = permutation_test(explanation_exact, explanation_approximate)

print(f'Observed difference: {observed_diff}')
print(f'p-value: {p_value}')
